<a href="https://colab.research.google.com/github/shivanshu1303/Simple-ML-Algos-Implemented/blob/main/Decision_Trees_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Here is another implementation of `Decision Trees`. The first one is coded for when the features take on binary(i.e. 0/1) values.
## Binary values however don't happen very often and most datasets we will deal with will require us to deal with features take on a wider or even continuous range of values.
This tree implementation would be more suited to that task.

### We of course start with importing the requred libraries

In [17]:
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Next, our implementation will actually use 2 different kinds of data structures. The 1st would be the tree itself but the 2nd would be one we would be one we use to represent the nodes in our tree.

The functions we use will also reflect that in their nomenclature i.e. the way they are named.

First, we do the node functions since there are only a couple of them and thus they are quite easy to complete(& to understand).

In [2]:
def node_initialization(feature_index=None,threshold_value=None,left_child_node=None,right_child_node=None,class_value=None):
  node={
      "feature":feature_index,
      "threshold":threshold_value,
      "left":left_child_node,
      "right":right_child_node,
      "value":class_value
  }
  return node

As we see above, we store each node as a dictionary. We also have to differentiate a leaf node i.e. a terminal node from other nodes. We obviously write a function to do that.

In [3]:
def is_leaf_node(node):
  return node["value"] is not None

Now, we're done with functions relating to nodes. We focus on the tree.

First, we just write down some global variables that we would use throughout our tree code so that we dont have to pass them over and over again.

These would be the 'stopping criteria' for our tree (i.e. the number of samples below which we will stop splitting & the maximum depth we will allow our tree to reach) and a few others that are mostly syntactic and implementation-related nitty-gritty.

In [4]:
min_samples_split=2
max_depth=10
n_features=None
root=None

Now, just like we would with a decision tree, we write the `fit` function to fit the tree to.

In [5]:
def fit_decision_tree(X,y,_min_samples_split=2,_max_depth=100,_n_features=None):
  global min_samples_split,max_depth,n_features,root
  min_samples_split=_min_samples_split
  max_depth=_max_depth
  n_features=_n_features
  root=grow_tree(X,y,0)

In the above code, you might have seen a new word - `global`. This only means that we want to use some variables that are 'global' variables and dont want to create new variables (that would be destroyed as soon as we exit the function).

After that, we do fairly standard assignment operations. Finally, we call the `grow_tree` fucntion, which as the name suggests is used to **grow** our tree.

In [6]:
def grow_tree(X,y,depth=0,min_samples_split=2,max_depth=100,n_features=None):
  n_samples, n_feats=X.shape

  n_labels=len(np.unique(y))

  # Check if we're already at the stopping limit using 3 conditions
  # * if the current depth is already equal to the max depth allowed
  # * if all the samples are of a single class(i.e. the samples are 100% pure)
  # * the number of samples are less than the minimum number of samples that we decided to stop
  # splitting on
  if (depth >= max_depth) or (n_labels == 1) or (n_samples <= min_samples_split):
    leaf_value=most_common_label(y)
    return create_node(value=leaf_value)

  feat_idxs = np.random.choice(n_feats,n_features if n_features else n_feats,replace=False)
  # Above, we choose what features to train our tree on. If we have been provided with 'n_features'
  # as an arguement to the function, we obviously use that but if we haven't then we use the n_feats
  # value

  # Now, we focus on finding the best split among all the splits possible
  # Also, for a particular feature, since it may not always be a binary one, we also have to
  # decide what threshold we choose i.e. where we draw a line for a particular feature's values
  best_feature, best_thresh = best_split(X,y,feat_idxs,min_samples_split)

  # Now that we know what the best feature to split is, we actually split the nodes
  left_idxs, right_idxs = split(X[:,best_feature],best_thresh)
  left=grow_tree(X[left_idxs,:],y[left_idxs],depth+1,min_samples_split,max_depth,n_features)
  right=grow_tree(X[right_idxs,:],y[right_idxs],depth+1,min_samples_split,max_depth,n_features)

  return create_node(feature=best_feature,threshold=best_thresh,left=left,right=right)

We now code the `create_node` function that would generate a node using the values that we passed to it.

In [7]:
def create_node(feature=None, threshold=None, left=None, right=None, value=None):
  return {"feature":feature, "threshold":threshold, "left":left, "right":right, "value":value}

After this, we code the `best_split` function we used in the `grow_tree` function that would provide us with the ideal feature to split on and the ideal threshold to divide samples

In [8]:
def best_split(X,y,feat_idxs,min_samples_split):
  best_gain=-1
  split_idx=None
  split_threshold=None

  for feat_idx in feat_idxs:
    X_column=X[:,feat_idx]
    thresholds=np.unique(X_column)
    # ie the 'threshold' we choose wont be derived by some mathematical calculation but only
    # from the values already existing in the set
    for threshold in thresholds:
      #gain=information_gain(y,X_column, threshold, min_samples_split)
      gain=information_gain(y,X_column, threshold)


      if gain>best_gain:
        best_gain=gain
        split_idx=feat_idx
        split_threshold=threshold

  return split_idx, split_threshold

This was a very simple implementation if we look at it. We just go feature by feature and for each feature, we try all possible values of the thresholds and just choose the best one. Kind of like a nested loop, if you will.

Next, we move on to calculate the `information_gain` function.

In [9]:
def information_gain(y,X_column, threshold):
  # Calculate the entropy of the parent
  parent_entropy=entropy(y)

  # Now, create children using the threshold value
  left_idxs, right_idxs=split(X_column, threshold)

  if((len(left_idxs)==0) or (len(right_idxs)==0)):
    return 0

  n=len(y)

  n_l=len(left_idxs)
  e_l=entropy(y[left_idxs])

  n_r=len(right_idxs)
  e_r=entropy(y[right_idxs])

  child_entropy=(n_l/n)*(e_l)+(n_r/n)*(e_r)

  return (parent_entropy - child_entropy)

Now, we're mostly done with the functionality.

Only the 'helper functions' remain. They help make the code look more readable by taking out the details so that we can focus better on the algorithm and not the syntactic nitty-gritty.

The first one among which is the `split` function that divides the node samples into 2 parts using their indices.

In [10]:
def split(X_column, split_thresh):
  left_idxs = np.argwhere(X_column<=split_thresh).flatten()
  right_idxs= np.argwhere(X_column >split_thresh).flatten()
  return left_idxs, right_idxs

Next, we go on to the `entropy` function

In [11]:
def entropy(y):
  hist=np.bincount(y)
  ps=hist/len(y)
  # The above provides us with the occurence ratio for each index where we interpret the indices
  # as class
  return -np.sum([p*np.log2(p) for p in ps if p>0])

In the above code, the really smart thing is the use of the `bincount` function. What this does is it returns the frequency of ocurence of each element in the array.

Please see the code-block below:

`np.bincount(np.array([1,2,3,1,2]))`

`>> array([0, 2, 2, 1])`

This can be interpreted as: `0` occurs zero times in the array, `1` and `2` occur two times in the array and `3` occurs 1 time in the array.

It returns the frequency of the index in the array. This works beautifully for us in helping us calculate probabilities.

The next function is the `most_common_label` function that would return use the most common label for a sample

i.e. per our model, the class of the label.

In [12]:
def most_common_label(y):
  counter=Counter(y)
  most_common=counter.most_common(1)[0][0]
  return most_common

The interesting part in the above function might be the amount of numbers we have to use in the right hand side of the line calculating `most_common`.

The 1st number i.e. `1` is the argument to the `most_common` method which tells it to return the 1 most common value in `y` i.e. the most frequently occuring value.

What it returns is a list. Each element of the list is a tuple.

The tuple has 2 elements:

* The first is the value in the argument passed to the counter.
* The second is the frequency of occurence of the value.

So, when we write `.most_common(1)[0][0]`, it returns the most common element as a tuple inside a list.

The 1st zero tells the program to access the first tuple and the 2nd zero tells it to access the 1st element in the tuple i.e. the most frequently occuring value.

Finally, we also do the `predict` helper function. This will provide us with results after we have trained our model.

In [13]:
def predict(X,root):
  return np.array([traverse_tree(x,root) for x in X])

Now, we do the `traverse_tree` function that would go down the tree and fetch a result value.

In [14]:
def traverse_tree(x,node):
  if is_leaf_node(node):
    return node["value"]

  if x[node["feature"]]<=node["threshold"]:
    return traverse_tree(x,node["left"])

  else:
    return traverse_tree(x,node["right"])

Now, we're done with everything the model might need.

We only have to add the accessory `accuracy` function so that we know our model's performance.

In [15]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

# We are done with the model

Now, we have to train it on data to see if it works well or not.

In [21]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


# def fit_decision_tree(X,y,_min_samples_split=2,_max_depth=100,_n_features=None)
fit_decision_tree(X_train, y_train, _min_samples_split=2, _max_depth=100, _n_features=None)


predictions = predict(X_test, root)

print(f"Accuracy: {100*accuracy(y_test, predictions)}")

Accuracy: 91.22807017543859


As we see above, on the `breast-cancer` dataset, we are able to correctly classify with a 92.11 percent accuracy.